In [1]:
import sys
import os
import pandas as pd
import json

#  path to src
sys.path.append(os.path.abspath("../../src"))

from noise import add_combined_noise
from train_bert import train_and_evaluate

metrics_list = []
noise_tag = "noise_40"

for run in range(3):
    print(f"Running {noise_tag} – repetition {run + 1}")

    #  cleaned dataset
    df = pd.read_csv("../../data/cleaning/2_cleaned_data.csv")

    #  relevant fields
    df["text"] = df[["sender", "receiver", "urls", "clean_text"]].fillna("").astype(str).agg(" ".join, axis=1)

    #  BERT input
    df = df[["text", "label"]].rename(columns={"text": "body"})

    # Apply 40% combined noise
    df["body"] = df["body"].apply(lambda x: add_combined_noise(x, noise_level=0.40, seed=42 + run))

    # Run 
    tag = f"{noise_tag}_r{run + 1}"
    train_and_evaluate(df, model_tag=tag, text_col="body")

    # Save metrics
    with open(f"results/metrics_{tag}.json") as f:
        metrics = json.load(f)
        metrics["noise_level"] = noise_tag
        metrics["run"] = run + 1
        metrics_list.append(metrics)

# Save 
df_metrics = pd.DataFrame(metrics_list)
df_metrics.to_csv(f"results/metrics_{noise_tag}.csv", index=False)
df_metrics


Running noise_40 – repetition 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 47.744, 'train_samples_per_second': 16.756, 'train_steps_per_second': 2.095, 'train_loss': 0.47786354064941405, 'epoch': 1.0}
Running noise_40 – repetition 2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 46.7249, 'train_samples_per_second': 17.121, 'train_steps_per_second': 2.14, 'train_loss': 0.43133556365966796, 'epoch': 1.0}
Running noise_40 – repetition 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 46.635, 'train_samples_per_second': 17.154, 'train_steps_per_second': 2.144, 'train_loss': 0.5094575881958008, 'epoch': 1.0}


,accuracy,f1_phishing,precision,recall,noise_level,run
0,0.930,0.9391,0.8852,1.0000,noise_40,1
1,0.915,0.9258,0.8760,0.9815,noise_40,2
2,0.920,0.9292,0.8898,0.9722,noise_40,3
